In [281]:
import pandas as pd
import numpy as np
path = '/home/alvaro/Escritorio/FIUBA/Programación/Organizacion de datos'
df= pd.read_csv(path + '/train.csv')

In [282]:
#Fuente: https://es.investing.com/currencies/usd-mxn-historical-data
path1 = '/home/alvaro/Descargas'
dolar= pd.read_csv(path1 + '/Datos históricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)

In [283]:
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)
dolar.head()

,price,year,month
0,18.7184,2018,9
1,19.0850,2018,8
2,18.6480,2018,7
3,19.9175,2018,6
4,19.9135,2018,5


In [284]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['year'] = df['fecha'].dt.year
df['month']= df['fecha'].dt.month

In [285]:
newDf= pd.merge(df, dolar, on=['year', 'month'], how='left')
newDf["dollar_price"] = newDf.apply(lambda row: row["precio"]/row["price"], axis=1)
newDf.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'year', 'month', 'price',
       'dollar_price'],
      dtype='object')

In [286]:
df = newDf.filter(["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                   'centroscomercialescercanos', 'dollar_price' ])
df["tipodepropiedad"].value_counts()

Casa                             141717
Apartamento                       57341
Casa en condominio                19297
Terreno                            9945
Local Comercial                    3055
Oficina comercial                  1741
Bodega comercial                   1406
Edificio                           1396
Terreno comercial                  1326
Casa uso de suelo                   708
Quinta Vacacional                   395
Duplex                              343
Villa                               340
Inmuebles productivos urbanos       200
Rancho                              170
Local en centro comercial           165
Departamento Compartido             141
Otros                               134
Nave industrial                      76
Terreno industrial                   31
Huerta                               20
Lote                                  5
Garage                                1
Hospedaje                             1
Name: tipodepropiedad, dtype: int64

In [287]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales

In [288]:
#Arreglando, no pueden haber propiedades con metros cubiertos pero sin metros totales
df['metrostotales'].fillna(0, inplace=True)
df['metrostotales'] = df.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
df.head()

,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,dollar_price
0,254099,Apartamento,2.0,1.0,2.0,Benito Juárez,Distrito Federal,80.0,80.0,23533.0,2015-08-23,0.0,0.0,0.0,0.0,0.0,135683.671398
1,53461,Casa en condominio,3.0,2.0,2.0,La Magdalena Contreras,Distrito Federal,268.0,180.0,24514.0,2013-06-28,0.0,0.0,0.0,1.0,1.0,278078.170864
2,247984,Casa,3.0,2.0,2.0,Tonalá,Jalisco,144.0,166.0,48551.0,2015-10-17,0.0,0.0,0.0,0.0,0.0,72727.272727
3,209067,Casa,2.0,1.0,1.0,Zinacantepec,Edo. de México,63.0,67.0,53666.0,2012-03-09,0.0,0.0,0.0,1.0,1.0,50753.494183
4,185997,Apartamento,2.0,1.0,1.0,Zapopan,Jalisco,95.0,95.0,47835.0,2016-06-07,0.0,0.0,0.0,0.0,0.0,62910.284464


In [289]:
#Filtrando el garage, lotes, hospedaje, otros
filtroTerreno = df["tipodepropiedad"].isin(["Huerta", "Nave industrial", "Terreno", "Terreno comercial", 
                                            "Bodega comercial", "Terreno industrial"])
dfTerreno = df[filtroTerreno]
dfTerreno.tipodepropiedad.value_counts()

Terreno               9945
Bodega comercial      1406
Terreno comercial     1326
Nave industrial         76
Terreno industrial      31
Huerta                  20
Name: tipodepropiedad, dtype: int64

In [290]:
#Dropeo todos los terrenos que tienen habitaciones ya que si las tienen no serian terrenos.
dfTerreno = dfTerreno[dfTerreno["habitaciones"].isna()]
dfTerreno.tipodepropiedad.value_counts()

Terreno               9467
Terreno comercial     1254
Bodega comercial      1249
Nave industrial         63
Terreno industrial      28
Huerta                  13
Name: tipodepropiedad, dtype: int64

In [291]:
#Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
#tener metros no cubiertos
filtro1 = df["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio",
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
dfCasas = df[filtro1]
dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
dfCasas.garages.fillna(0, inplace=True)
dfCasas.isnull().any()

id                            False
tipodepropiedad               False
habitaciones                  False
garages                       False
banos                         False
ciudad                         True
provincia                      True
metroscubiertos               False
metrostotales                 False
idzona                         True
fecha                         False
gimnasio                      False
usosmultiples                 False
piscina                       False
escuelascercanas              False
centroscomercialescercanos    False
dollar_price                  False
dtype: bool

In [292]:
df = pd.merge(dfCasas, dfTerreno, on=['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos'
                                      ,'ciudad','provincia','metrostotales','idzona','fecha', 'gimnasio','usosmultiples',
                                       'piscina','escuelascercanas','centroscomercialescercanos', 'dollar_price'], how='outer')
df.columns

Index(['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad',
       'provincia', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'dollar_price'],
      dtype='object')

In [299]:
df.shape

(213028, 17)

In [296]:
df.tipodepropiedad.value_counts()

Casa                  127228
Apartamento            54955
Casa en condominio     17936
Terreno                 9467
Terreno comercial       1254
Bodega comercial        1249
Casa uso de suelo        386
Quinta Vacacional        321
Rancho                   128
Nave industrial           63
Terreno industrial        28
Huerta                    13
Name: tipodepropiedad, dtype: int64